In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
from simulators.cosmic_shear_map.cosmic_shear import *
import ndes.ndes as ndes
import delfi.delfi as delfi
import distributions.priors as priors
import time
import compression.score.score as score
import tensorflow as tf
import IMNN.IMNN as IMNN
import IMNN.ABC.ABC as ABC
%matplotlib inline

In [2]:
# Set up...

# Redshift distributions
pz_fid = pickle.load(open('simulators/cosmic_shear_map/pz_5bin.pkl', 'rb'))
nz = len(pz_fid)

# Fiducial parameters about which data compression is performed
theta_fiducial = np.array([0.3, 0.8, 0.05, 0.70, 0.96])

# Set up the truncated Gaussian prior...

# Prior boundaries
lower = np.array([0, 0.4, 0, 0.4, 0.7])
upper = np.array([1, 1.2, 0.1, 1.0, 1.3])

# Prior mean and covariance
prior_mean = np.array([0.3, 0.8, 0.05, 0.70, 0.96])
prior_covariance = np.eye(5)*np.array([0.1, 0.1, 0.05, 0.3, 0.3])**2

# Create prior over parameters
prior = priors.TruncatedGaussian(prior_mean, prior_covariance, lower, upper)

# Parameter names and ranges for plotting...  
names = ['\Omega_m', 'S_8', '\Omega_b', 'h', 'n_s']
labels =  ['\\Omega_m', 'S_8', '\\Omega_b', 'h', 'n_s']
ranges = {'\Omega_m':[lower[0], upper[0]], 'S_8':[lower[1],upper[1]],'\Omega_b':[lower[2],upper[2]],
            'h':[lower[3],upper[3]],'n_s':[lower[4],upper[4]]}

# Simulation set-up...

# Resolution
nside = 128
lmax = 3*nside-1
lmin = 10
n_ell_bins = 7
npix = hp.nside2npix(nside)

# Mask (Euclid)
mask = hp.read_map('simulators/cosmic_shear_map/Euclid_footprint_and_StarMask512.fits')
if nside != 512:
    mask = hp.pixelfunc.ud_grade(mask, nside)
mask = np.array([mask for x in range(nz)])

# Pixel-space shape noise standard deviation at each redshift
sigma_e = 0.3
n_p_mean = 1.6e9/sum(mask)
sig_n_p = sigma_e**2/np.random.poisson(n_p_mean, size=(nz, npix))

# Simulation and simulation arguments
def simulator(theta, seed, simulator_args, batch):
    return simulate(theta, seed, simulator_args, batch)[0,:]
simulator_args = [pz_fid, lmin, lmax, sig_n_p, mask, n_ell_bins]

# Simulate data (and time it for benchmarking)
start = time.time()
data = simulator(theta_fiducial, 0, simulator_args, 1)
print(time.time()-start)

NSIDE = 512
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
3.756875991821289


In [ ]:
### SET UP THE COMPRESSOR ###

input_shape = [nz*n_ell_bins*(n_ell_bins+1)//2]

# Fiducial parameters
θ_fid = np.array([0.3, 0.8, 0.05, 0.70, 0.96])
Δθpm = θ_fid*0.05

# Compute the mean and covariance
n_sims_for_covariance = 2000
Compressor = score.Gaussian(len(data), theta_fiducial, prior_mean = prior_mean, prior_covariance = prior_covariance)
Compressor.compute_mean_covariance(simulator, n_sims_for_covariance, simulator_args = simulator_args)
sims = Compressor.simulations

# Fiducial sims (and validation ones)
IMNN_data = {"data": sims[0:1000,:], "validation_data": tt[1000:,:]}

# Compute the derivatives
n_sims_for_derivatives = 100
Compressor.compute_derivatives(simulator, n_sims_for_derivatives, Δθpm, simulator_args = simulator_args)

# Derivative sims (incl for validation case)
IMNN_data["data_d"] = np.array([Compressor.dmudt])
IMNN_data["validation_data_d"] = np.array([Compressor.dmudt])

In [ ]:
parameters = {
    "dtype": 32,
    "number of simulations": 1000,
    "number of derivative simulations": 1,
    "fiducial": θ_fid.tolist(),
    "number of summaries": 5,
    "input shape": input_shape,
    "filename": "data/model",
}

tf.reset_default_graph()
n = IMNN.IMNN(parameters = parameters)

def build_network(data, **kwargs):
    α = kwargs["activation_parameter"]
    input_shape = kwargs["input_shape"]
    with tf.variable_scope("layer_1"):
        weights = tf.get_variable("weights", shape = [input_shape[-1], 128], initializer = tf.variance_scaling_initializer())
        output = tf.nn.leaky_relu(tf.matmul(data, weights, name = "multiply"), name = "output")
    with tf.variable_scope("layer_2"):
        weights = tf.get_variable("weights", shape = (128, 128), initializer = tf.variance_scaling_initializer())
        output = tf.nn.leaky_relu(tf.matmul(output, weights, name = "multiply"), name = "output")
    with tf.variable_scope("layer_3"):
        weights = tf.get_variable("weights", shape = (128, 128), initializer = tf.variance_scaling_initializer())
        output = tf.nn.leaky_relu(tf.matmul(output, weights, name = "multiply"), name = "output")
    with tf.variable_scope("layer_4"):
        weights = tf.get_variable("weights", shape = (128, n.n_summaries), initializer = tf.variance_scaling_initializer())
        output = tf.matmul(output, weights, name = "output")
    return output

α = tf.placeholder(dtype = tf.float32, shape = (), name = "activation_parameter")
network = lambda x: build_network(x, activation_parameter = α, input_shape = input_shape)
graph_dictionary = {"activation_parameter:0": 0.001}

n.setup(network = network, load_data = IMNN_data)

In [ ]:
# Train the IMNN
updates = 10000
at_once = 1000
learning_rate = 1e-4

n.train(updates, at_once, learning_rate, constraint_strength = 2.,
        training_dictionary = graph_dictionary,
        validation_dictionary = graph_dictionary,
        get_history = True, restart = False, diagnostics = False)

In [ ]:
# Plot the IMNN training curve
fig, ax = plt.subplots(1, 1, sharex = True, figsize = (10, 6))
epochs = np.arange(1, len(n.history["det F"]) + 1)
ax.plot(epochs, n.history["det F"], label = 'Training data')
ax.plot(epochs, n.history["det test F"], label = 'Test data')
ax.legend(frameon = False)
ax.set_xlim([1, epochs[-1]]);